### Notebook to plot and organize CuteChess metrics form a nni experiment
<br>Files to run the NNI-Experiment are expected to be in the same directory<br />
<br>tournament_loop.py : main loop<br />
<br>tournament_config.py : config used for the Cute Chess Tournament<br />
<br>config.yml : config starting the NNI experiment<br />
<br>search_space.json : declaring the Search Space for the hyperparameter tuner<br />

In [ ]:
import time
nni_config = r"C:\Users\Daniel\MPVCrazyAra\CrazyAra\nni_tournament\config.yml"
search_space = r"C:\Users\Daniel\MPVCrazyAra\CrazyAra\nni_tournament\search_space.json"

## engine names  (elo diff from contender perspective)
baseline_name = "CrazyAra_Standard"
contender_name = "MPV_Large_Standard"##"CrazyAra_RISEv2_27_blocks"

start_experiment_from_notebook = False ## assumes no experiment is currently running
nni_refresh_rate = 30 ## refresh rate of experiment in seconds
plot_metric = "LargeNet_Policy_Weight"

from pprint import pprint
import numpy as np
from tournament_config import tournament_config
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
import glob
import os
import configparser
import re
from pathlib import Path
import ast
import json

In [ ]:
if start_experiment_from_notebook:
    !nnictl create --config $nni_config

    try:
        experiment = !nnictl experiment show
        experiment = (''.join(experiment)).replace(" ","")
        experiment = json.loads(experiment)

        experiment_path = experiment["logDir"]
        experiment_id = experiment["id"]

        experiment_status = "RUNNING"

        while experiment_status == "RUNNING" or experiment_status == "INITIALIZED":
            time.sleep(nni_refresh_rate)
            experiment_status = !nnictl experiment status
            experiment_status = (''.join(experiment_status)).replace(" ","")
            experiment_status = json.loads(experiment_status)["status"]

        if experiment_status != "FINISHED": #in case of error / no more tuner trials
            !nnictl stop $experiment_id

    except Exception as exception:
        !nnictl stop $experiment_id
        raise exception

else:
    ## path to experiment directory
    experiment_path = r"C:\Users\Daniel\nni-experiments\n1FUHlwO"

In [ ]:
trials_path = experiment_path + r"\trials"

Cute Chess Tournament config

In [ ]:
pprint(tournament_config)

In [ ]:
##dict containing all trials with key of the fixed_movetime
experiment_dict = {}

plot_metrics = list()
## iterate over all trials
parser = configparser.ConfigParser()
for trial in os.scandir(trials_path):
    parameter_file = open(Path(trial.path + r"\parameter.cfg"))
    metric_file = open(Path(trial.path + r"\.nni\metrics"))
    parameter = json.loads(parameter_file.read())['parameters']
    test = metric_file.read()
    metric = re.findall(r"[{][\s\S]*[}]", test)

    if metric:
        metric = metric[0]
        metric = ast.literal_eval(metric)
        metric = json.loads(metric['value'])

    ## configure dict for trial, containing metrics (elo+var) and settings
    trial_dict = {}
    trial_dict.update(parameter)
    if metric:
        trial_dict['elo'] = float(metric['elo'])
        trial_dict['variance'] = float(metric['variance'])
    else:
        trial_dict['elo'] = 'inf'
        trial_dict['variance'] = 'nan'

    plot_metrics.append(parameter[plot_metric])


    if parameter[plot_metric] not in experiment_dict:
        experiment_dict[parameter[plot_metric]] = list()
    experiment_dict[parameter[plot_metric]].append(trial_dict)
## add plot metrics to the dict
experiment_dict[plot_metric] = plot_metrics
experiment_dict['CuteChessConfig'] = tournament_config

Configure Plot Settings

In [ ]:
print(plt.style.available)
# plt.style.use('seaborn-paper')
plt.style.use('grayscale')
plt.style.use('seaborn-whitegrid')
# plt.style.use('seaborn-white')
#plt.style.use('fivethirtyeight')
rc('font', **{'family': 'serif'})  # ,'sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
# rc('font',**{'family':'serif','serif':['Palatino']})
# rc('text', usetex=True)
# print(plt.rcParams.keys())
plt.rcParams['legend.frameon'] = 'True'
plt.rcParams['legend.framealpha'] = '1.0'
print(plt.rcParams['figure.figsize'])
plt.rcParams['figure.figsize'] = [6.4*0.7, 4.8*0.7] #(15, 5)
plt.rcParams['axes.grid.axis'] = 'y' #False
# https://matplotlib.org/3.1.0/gallery/lines_bars_and_markers/linestyles.html
linestyle_str = [
  ('solid', 'solid'),  # Same as (0, ()) or '-'
  ('dotted', 'dotted'),  # Same as (0, (1, 1)) or '.'
  ('dashed', 'dashed'),  # Same as '--'
  ('dashdot', 'dashdot')]  # Same as '-.'
linestyle_tuple = [
  ('loosely dotted', (0, (1, 10))),
  ('dotted', (0, (1, 1))),
  ('densely dotted', (0, (1, 1))),

  ('loosely dashed', (0, (5, 10))),
  ('dashed', (0, (5, 5))),
  ('densely dashed', (0, (5, 1))),

  ('loosely dashdotted', (0, (3, 10, 1, 10))),
  ('dashdotted', (0, (3, 5, 1, 5))),
  ('densely dashdotted', (0, (3, 1, 1, 1))),

  ('dashdotdotted', (0, (3, 5, 1, 5, 1, 5))),
  ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
  ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1)))]


Prepare Data for Plot

In [ ]:
x = experiment_dict[plot_metric]
x.sort()
elo =  list()
variance = list()

for timestamp in x:
    elo_tmp = []
    var_tmp = []
    settings = experiment_dict[timestamp]
    for plot_setting in settings:
        elo_tmp.append(float(plot_setting['elo']))
        var_tmp.append(float(plot_setting['variance']))
    elo.append(elo_tmp)
    variance.append(var_tmp)

elo = np.column_stack(elo).tolist()
variance = np.column_stack(variance).tolist()

## create Plot label
plot_label =  list()
for param in experiment_dict[x[0]]:
    str_tmp = ""
    for key in param.keys():
        if key != plot_metric and key != 'elo' and key != 'variance': ## already stated
            next_arg = ""
            if isinstance(param[key],str):
                if param[key] == 'true':
                    next_arg = key
            else: ## param is float (or int)
                next_arg =  key + " = " + str(param[key])
            if next_arg:
                if not str_tmp:
                    str_tmp = next_arg
                else:
                    str_tmp = str_tmp + "; " + next_arg
    plot_label.append(str_tmp)
## add elo and var to experiment dict for better access later
experiment_dict['plot_data'] = {'x': x, 'elo': elo, 'var': variance, 'label': plot_label, 'contender_name': contender_name, 'baseline_name': baseline_name}

Plot Experiment

In [ ]:
figures = list()
plot_data = experiment_dict['plot_data']
for elo_plt, var_plt, label in zip(np.array(plot_data['elo']), np.array(plot_data['var']), np.array(plot_data['label'])):
    fig = plt.figure()
    fig.suptitle("{} vs. {}".format(plot_data['contender_name'], plot_data['baseline_name']))
    plt.grid(True)

    if label:
        fig.text(0, -0.05, "Additional Settings:")
        fig.text(0,-0.10,label)

    plt.xticks(x)
    plt.xlabel(plot_metric)
    plt.ylabel("Relative elo difference")
    plt.plot(x, elo_plt, label="elo difference between: \n {} \n {}".format(contender_name, baseline_name))
    plt.fill_between(x, elo_plt - var_plt, elo_plt + var_plt, color='black', alpha=0.25)
    plt.legend()
    plt.show()

    figures.append(fig)

Save Plots + Data as dictionary

In [ ]:
plot_data_dir = experiment_path + r"\plot_data"
if not os.path.isdir(plot_data_dir):
    os.mkdir(plot_data_dir)

with open(plot_data_dir + r'\experiment_data.json', 'w') as file:
    json.dump(experiment_dict, file, indent=len(x))

for fig, plot_num in zip(figures,  np.arange(len(figures))):
    fig.savefig(plot_data_dir + r"\{}_vs_{}_{}.pdf".format(contender_name, baseline_name, plot_num), bbox_inches="tight")
